In [102]:
!pip install names

     |████████████████████████████████| 798kB 1.3MB/s eta 0:00:01
  Created wheel for names: filename=names-0.3.0-cp37-none-any.whl size=803689 sha256=136e5bc591c7b09d7ec5ee4eb046b545e8e689dcdba7a08277d6fd2d33ca7121
  Stored in directory: /Users/chazarey/Library/Caches/pip/wheels/f9/a5/e1/be3e0aaa6fa285575078fa2aafd9959b45bdbc8de8a6803aeb
Successfully built names
You should consider upgrading via the 'pip install --upgrade pip' command.


In [103]:
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import uuid
import os
from os import path
from glob import glob
import json
import re
import pickle
import argparse

import numpy as np
from textwrap import wrap
from scipy import ndimage, misc

#import config
#from type import RecipeContainer, DataContainer
#from utils import url_to_filename

# import matplotlib using agg backend
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import elasticsearch
import boto3
import requests
import random
import names

{name: “Heinz”, craft:”butcher”, language:”de”, ...}
{name: “Giorgio”, craft:”baker”, language:”it”, ...}

In [30]:
#https://www.foodstandards.gov.au/science/monitoringnutrients/ausnut/ausnutdatafiles/pages/foodnutrient.aspx

food_ing_df = pd.read_csv('recipes/FoodNutrientDatabase.csv')
print(food_ing_df.shape)
food_ing_df.head()

(5740, 57)


,Food ID,Survey ID,Food Name,Survey flag,"Energy, with dietary fibre (kJ)","Energy, without dietary fibre (kJ)",Moisture (g),Protein (g),Total fat (g),"Available carbohydrates, with sugar alcohols (g)",...,Total saturated fat (g),Total monounsaturated fat (g),Total polyunsaturated fat (g),Linoleic acid (g),Alpha-linolenic acid (g),C20:5w3 Eicosapentaenoic (mg),C22:5w3 Docosapentaenoic (mg),C22:6w3 Docosahexaenoic (mg),Total long chain omega 3 fatty acids (mg),Total trans fatty acids (mg)
0,10F40019,31103001,"Beef, extract, bonox",NaN,401,401,56.6,16.6,0.2,6.5,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
1,13A12001,31302001,"Basil, dried",NaN,1079,774,10.0,18.2,5.5,15.5,...,2.36,1.40,0.61,0.26,0.34,0.0,0.0,0.0,0.0,0.0
2,10E10113,31302002,"Cardamom, seeds, ground",NaN,1333,1109,8.3,10.8,6.7,40.5,...,2.20,2.81,1.39,1.00,0.39,0.0,0.0,0.0,0.0,0.0
3,10E10098,31302003,Chilli (chili) powder,NaN,1441,1167,7.8,12.3,16.8,20.5,...,2.41,3.15,7.85,7.32,0.52,0.0,0.0,0.0,0.0,0.0
4,10E10107,31302004,"Chilli (chili), dried, ground",NaN,1573,1355,6.6,14.1,16.8,29.6,...,2.42,3.16,7.89,7.36,0.52,0.0,0.0,0.0,0.0,0.0


In [59]:
i = 0 
unique_words = set()
for index, row in food_ing_df.iterrows():
    i += 1
    #print(row['Food Name'])
    row_data = row['Food Name']
    unique_words.update(row_data.lower().split())

#row['Food Name'].str.lower().str.split().apply(unique_words.update)

print(unique_words)

{'varieties),', 'middle', 'high', 'shortening,', 'heated', 'meatloaf,', 'flavoured,', 'g)', 'acids,', 'up', 'groats,', 'any', 'deep-fried,', 'strawberry,', 'lecithin,', '(~3.5%)', '70%,', 'camel,', 'dim', 'guacamole', '(rice', 'subway', 'unfilled', 'making', '(tom', '(easy', 'lebanese,', 'cheesecake,', '(milo', '(chicken', 'sultanas),', 'doughnut,', 'raspberry', 'chup', 'dinner,', 'sunflower,', 'ball', 'royal,', '70%', 'alfalfa', 'macchiato,', 'food', '(hundreds', 'nugget', 'stewed', 'macadamias,', 'broth', 'formulation', 'identified', 'cardamom,', 'promite', 'unprocessed,', '(inari)', 'meat', 'content', 'puffed', 'dog', 'marshmallow,', 'frozen', '465', '(as', 'cereal', 'trifle,', 'rum,', 'syrup', 'palm', 'iced', 'powdered', 'crust,', 'extracts', 'marjoram,', 'beans,', 'a,', 'gum,', 'buckwheat', 'quandong,', 'soda,', '(~3%)', 'pure', 'chapatti,', 'sugar-glazed', 'nog)', 'used', 'in', 'sherry,', 'cows', 'soda', 'moreton', '(~6%),', 'kingfish,', 'above)', 'quinoa', 'breadcrumb', 'passion

In [67]:
random.sample(unique_words, 50)

['dog',
 'gala,',
 'shrimp',
 'caffeinated',
 '4%',
 'round',
 'parboiled',
 'avocado,',
 'grouper,',
 'sin,',
 'stable,',
 '(~25%)',
 'friand,',
 'sage,',
 'tarts',
 'soy',
 'lima,',
 'kiwifruit',
 'condensed,',
 'defined',
 'cultured,',
 'herbal,',
 'fromais',
 'fruit-filled,',
 'nutrition,',
 'use,',
 'cow,',
 'tea,',
 'iced,',
 'quinoa',
 'style,',
 'any',
 'marmite',
 'celeriac,',
 'sauce',
 'coffee)',
 'honeycomb',
 'high',
 'organs,',
 'spinach,',
 '(100%',
 'marinated,',
 'a2',
 '(inari)',
 'added,',
 'cannellini,',
 'cream-filled,',
 'halvah,',
 'ricotta,',
 'bartlett,']

In [83]:
recipes_df = pd.read_json('recipes/bulk_openrecipes.json', orient='rows', lines=True)
recipes_df.head()
i = 0 
unique_ing_words = set()
for index, row in recipes_df.iterrows():
    i += 1
    #print(row['ingredients'])
    row_data = row['ingredients']
    if(isinstance(row_data, str)):
        unique_ing_words.update(row_data.lower().split())

#row['Food Name'].str.lower().str.split().apply(unique_words.update)
print(unique_ing_words)

{'4-5', 'ricotta-stuffed', 'middle', 'jalapeño,', 'cupcakes.', 'shortening,', 'process', 'before', 'dogs', 'fingers', 'heated', '(good)', '6+', 'cored,', 'chunky', '(thai', 'soba', '(400g)', '(10-15', 'g)', 'pit', 'either', 'shallot', 'save', 'powder*', 'up', 'lukewarm', 'cotija', 'shoyu,', '(~2', 'bunch', 'pepperoncini', 'any', 'reserve', 'meat:', 'sugar-mashed', 'dot', 'combined.', 'flat-leaf', 'bundt', 'smallish', 'unskinned', 'sticks', '(chiffonade)', "(don't", 'strawberries,', 'mustard)', 'dressing:', 'nero', 'biga', 'berries),', 'scallions),', 'tuck', 'enough', 'guacamole', 'fettuccine', '70g', 'mushrooms)', 'shoots,', 'spearmint', 'doughnuts', 'pan', '225', 'them),', 'bit-sized', 'jamaica', 'good,', 'must', 'savoy', 'them.', '450g', 'urid', '500', 'radishes', 'slits', 'thinly,', '(2%', 'counter.', 'cubes.', 'dough"', 'pepitas)', 'needed,', 'pan-frying', 'raspberry', 'parmigiano-reggiano', 'horseradish', '(quantities', 'cajun', 'penne', 'x', 'sage', '70%', 'nonalkalized', 'right'

In [94]:
type(random.sample(unique_words, 50)) #TODO - cleanup non-letter charecters

list

In [99]:
', '.join(random.sample(unique_words, 50))

"sizes, dusting, kosher), cups),, labeled, (brown, tiny-inch, (more,, serrano), mustard,, choice:, (720g), (quantities, eye, (parmesan, tortillas:, bulgar, melon), fava, pea, (31/2, paper-thin, 425f, feeling., (store, breast,, 285, blackberry, sauteed, cubed, crème, chile,, /120, chops:, apple), crushed, lima, removes,, fruits,, create, serrano,, parsley, jam, (1, grater, (frank's,, together, (coating, berries, until"

In [64]:
i = 1
for i in range(2):
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    sw_people_json = json.loads(r.content)
    print(sw_people_json)
    if 'name' in sw_people_json:
        print(sw_people_json['name'])
    i=i+1


{'detail': 'Not found'}
{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.co/api/planets/1/', 'films': ['https://swapi.co/api/films/2/', 'https://swapi.co/api/films/6/', 'https://swapi.co/api/films/3/', 'https://swapi.co/api/films/1/', 'https://swapi.co/api/films/7/'], 'species': ['https://swapi.co/api/species/1/'], 'vehicles': ['https://swapi.co/api/vehicles/14/', 'https://swapi.co/api/vehicles/30/'], 'starships': ['https://swapi.co/api/starships/12/', 'https://swapi.co/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.co/api/people/1/'}
Luke Skywalker


In [65]:
sw_people_json['name']

'Luke Skywalker'

In [86]:
zip_codes_df = pd.read_csv('zip/zip_codes.csv')
print(zip_codes_df.shape)
zip_codes_df.head()

(100, 3)


,ZIP_CODE,CITY,INCOME
0,79936,El Paso TX,42857
1,90011,Los Angeles CA,23851
2,60629,Chicago IL,40279
3,90650,Norwalk CA,46012
4,90201,Bell Gardens CA,30029


In [106]:
import time

NUM_USERS  = 500

_occupations = ['butcher', 'baker', 'deli', 'pastry', 'chef', 'sue chef', 'candy maker']
_occupations_len = len(_occupations)
_languages = ['en', 'de', 'it', 'in', 'gu', 'ka', 'el']
_languages_len = len(_languages)
_food_items = unique_ing_words

def gen_users(num_users):
    _users = []
    _zip_len = zip_codes_df.shape[0]
    
    for i in range(num_users):
        _age        = int(np.random.normal(40, 4))
        _gender_idx = np.random.randint(0, 2)
        _loc_idx    = np.random.randint(0, _zip_len)
        _occupation_user = np.random.randint(0, _occupations_len)
        _languages_selected = np.random.randint(0, _languages_len)
        _food_items_selected = np.random.randint(0, _food_items_len)
        
        if _gender_idx == 0:
            _gender = 'MALE'
            _name = names.get_full_name(gender='male')
        else:
            _gender = 'FEMALE'
            _name = names.get_full_name(gender='female')
            
        _this_user = [str(i), _age, _gender, 
                      str(zip_codes_df.iloc[_loc_idx][0]), 
                      zip_codes_df.iloc[_loc_idx][2],
                      _occupations[_occupation_user],
                      _languages[_languages_selected],
                      ', '.join(random.sample(_food_items, 50)),
                      _name]
        
        _users.append(_this_user)
        
    _df = pd.DataFrame(_users, columns=['USER_ID', 'AGE', 'GENDER', 'LOCATION', 'SALARY', 'OCCUPATION', 'LANGUAGE', 'FAV', 'NAME'])
    return _df

users_df = gen_users(NUM_USERS)
print(users_df.shape)

users_df.to_csv('users/food_ind_generated_users.csv', index=False)

(500, 9)


In [107]:
users_df

,USER_ID,AGE,GENDER,LOCATION,SALARY,OCCUPATION,LANGUAGE,FAV,NAME
0,0,41,FEMALE,90650,46012,baker,it,"shortening, menthe, booze, head, (organic), sw...",Mary Devoll
1,1,37,FEMALE,93550,37484,pastry,it,"grains), times,, pressed, tempeh,, {4, broiler...",Sheila Cantara
2,2,46,MALE,90255,30375,chef,gu,"oiling, guacamole:, (3.8, onion, raddiccio,, ""...",Matthew Wolfgram
3,3,41,FEMALE,23464,53486,chef,de,"etc), turbinado, jelly,, approx., 170g, waterm...",Anita Britten
4,4,43,MALE,93065,72384,baker,gu,"amelie's-provided, asparagus,, drizzling, 25, ...",Jimmie Smith
...,...,...,...,...,...,...,...,...,...
495,495,40,MALE,11235,31013,baker,ka,"560, (other, skinless, (6-ounce), (swiss,, (wa...",John Wells
496,496,39,FEMALE,30043,71424,baker,en,"salsa, pestle, cream,, tsp, whiz, sage, pickle...",Wanda Singleton
497,497,39,FEMALE,30349,39141,pastry,de,"pepperoni, lb.),, curls, bass,, like, somewhat...",Elizabet Fletcher
498,498,43,FEMALE,91911,38010,deli,en,"packed, glaze:, dulse, olives:, seconds,, stro...",Nida Simcoe
